In [5]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from typing import TypedDict

In [6]:
load_dotenv()

True

In [7]:
model = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    temperature=0
)

In [8]:
# Create a state
class LLMState(TypedDict):

    question: str
    answer: str

In [10]:
def llm_qa(state: LLMState)->LLMState:

    # extract the question from state
    question=state['question']

    # form a prompt
    prompt=f'Answer the following question {question}'

    # ask that question to the LLM
    answer=model.invoke(prompt).content

    #update the answer in the state
    state['answer']=answer 

    return state

In [12]:
# Create our graph
graph =StateGraph(LLMState)

# Add nodes
graph.add_node('llm_qa',llm_qa)

# Add Edge
graph.add_edge(START,'llm_qa')
graph.add_edge('llm_qa',END)

# Compile
workflow=graph.compile()

In [14]:
# Execute 
initial_state={'question':'How far from the moon from the earth'}
final_state=workflow.invoke(initial_state)
print(final_state)

{'question': 'How far from the moon from the earth', 'answer': "The distance between the Earth and the Moon is not constant because the Moon's orbit around the Earth is elliptical, not a perfect circle.\n\nHere are the key figures:\n\n*   **Average Distance:** Approximately **384,400 kilometers** (238,900 miles). This is the most commonly cited figure.\n*   **Closest Point (Perigee):** When the Moon is closest to Earth, the distance can be around **363,104 kilometers** (225,623 miles).\n*   **Farthest Point (Apogee):** When the Moon is farthest from Earth, the distance can be around **405,696 kilometers** (252,088 miles).\n\nSo, while the average is a good general answer, the actual distance varies throughout the month."}
